<a href="https://colab.research.google.com/github/hamed1337/Virtual-Assistant/blob/master/wavenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports and constants

In [0]:
tensorflow2 = False

In [0]:
if tensorflow2:
  !pip uninstall keras
  !pip uninstall tensorflow
  !pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
#import utils
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
# from tensorflow.keras.backend import manual_variable_initialization
# manual_variable_initialization(False)

from IPython.core.debugger import Tracer
import time
from tensorflow.keras.models import Model
import numpy as np
import pickle 

from tensorflow.keras.layers import Conv1D, BatchNormalization, Lambda, concatenate, Flatten, Input, Dense
from tensorflow.keras.models import load_model

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import signature_constants, tag_constants
from tensorflow.python.saved_model.signature_def_utils_impl import predict_signature_def

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

import gc

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# from tensorflow.python.client import device_lib

# device_lib.list_local_devices()

In [0]:
batch_size = 16
input_dimension = 20 #MUST BE CHANGED TO MATCH MFCC COEFFICIENTS DIMENSION
num_dim = 128      # latent dimension
num_blocks = 3
output_dimension = 29
epochs = 20

In [0]:
root_path = '/content/drive/My Drive/Data/VoiceData/'
model_path = '/content/drive/My Drive/Data/'
libri_path = root_path + 'Libri/'
ted_path = root_path + 'Tedlium/'
vctk_path = root_path + 'Vctk/'
number_of_files = 3
data_paths = [vctk_path, ted_path, libri_path]
genders = ['m', 'f']

#Reading the data (this section should be in a separate .py file)

In [0]:
def combine_files(files):
  return np.concatenate([np.load(file, allow_pickle = True) for file in files])

In [0]:
#returns the inputs length
def get_input_length_numpy(inputs):
    return [np.count_nonzero(_input.sum(axis = -1), axis = -1) for _input in inputs]

In [0]:
def batch_data(l, padding_value = 0):
  return [pad_sequences(l[i : i +  batch_size], padding = 'post', value = padding_value) for i in range(0, len(l), batch_size)]

In [0]:
def expand_labels(batch_labels):
   return [np.expand_dims(item, axis = -1) for item in batch_labels]

In [0]:
def read_data(mfccs_paths, labels_paths):
  mfcc_np = combine_files(mfccs_paths)
  label_np = combine_files(labels_paths)
  print(mfcc_np.shape)
  print(label_np.shape)
  return train_test_split(mfcc_np, label_np, test_size = 0.05, random_state = 42)

In [0]:
def get_data(mfccs_paths, labels_paths):
  data = read_data(mfccs_paths, labels_paths)
  
  padding_values = [0, 0, -1, -1]
  
  data_batches = [batch_data(data[i], padding_values[i]) for i in range(len(data))]
      
  X_train_batches, X_test_batches, Y_train_batches, Y_test_batches = tuple(data_batches)
  
  input_length_train = get_input_length_numpy(X_train_batches)
  input_length_test = get_input_length_numpy(X_test_batches)
  
  Y_train_batches = expand_labels(Y_train_batches)
  Y_test_batches = expand_labels(Y_test_batches)
  
  return X_train_batches, Y_train_batches, input_length_train, X_test_batches, Y_test_batches, input_length_test

In [0]:
def get_all_data():
  
  import itertools
  
  mfccs_paths = [[path + 'mfccs_' + gender + '_' + str(i) + '.npy'  for gender, path in itertools.product(genders, data_paths)] for i in range(number_of_files)]
  labels_paths =  [[path + 'labels_' + gender + '_' + str(i) + '.npy'  for gender, path in itertools.product(genders, data_paths)] for i in range(number_of_files)]
  
  all_data = [np.asarray([]) for i in range(6)]
  
  for mfccs, labels in zip(mfccs_paths, labels_paths):
    data = get_data(mfccs, labels)
    for i in range(len(data)):
      all_data[i] = np.concatenate((all_data[i], data[i]))
      
  return tuple(all_data)

In [0]:
X_train_batches, Y_train_batches, input_length_train, X_test_batches, Y_test_batches, input_length_test = get_all_data()

(66784,)
(66784,)
(66784,)
(66784,)
(66777,)
(66777,)


In [0]:
Y_train_batches.shape

(11897,)

In [0]:
gc.collect()

10

#Model

In [0]:
def res_block(tensor, size, rate, block, dim = num_dim):

    # filter convolution
    conv_filter = Conv1D(num_dim, size, activation='tanh', dilation_rate = rate, padding = 'causal')(tensor)
    conv_filter = BatchNormalization()(conv_filter)

    # gate convolution
    conv_gate = Conv1D(num_dim, size, activation = 'sigmoid', dilation_rate = rate, padding = 'causal')(tensor)
    conv_gate = BatchNormalization()(conv_gate)

    # output by gate multiplying
    out = keras.layers.multiply([conv_filter,conv_gate])

    # final output
    out = Conv1D(num_dim, 1, activation='tanh', padding='same')(out)
    out = BatchNormalization()(out)

    # residual and skip output
    return keras.layers.add([out, tensor]), out

In [0]:
def create_model():
  
  tf.reset_default_graph()
  sess = tf.Session()
  K.set_session(sess)
   
  inputs = Input(shape = (None, input_dimension,), name = 'inputs')
  z = Conv1D(num_dim, 1, activation = 'tanh', padding = 'same')(inputs)

  skip = []
  for i in range(num_blocks):
      for r in [1, 2, 4, 8, 16]:
          z, s = res_block(z, size = 7, rate = r, block = i)
          skip.append(s)

  skip = keras.layers.add(skip)
  
  logit = Conv1D(num_dim, 1, activation = 'tanh', padding = 'same')(skip)
  logit = BatchNormalization()(logit)

  logit = Conv1D(output_dimension, 1, padding = 'same', activation = 'linear')(logit)
  
  return inputs, logit

In [0]:
def get_input_length(inputs):
  return tf.reduce_sum(tf.cast(tf.not_equal(tf.reduce_sum(inputs, axis = -1), 0.), 'int32'), axis = -1)

In [0]:
def get_tensor_lengths(tensor, padding_value):
  return tf.cast(tf.reduce_sum(tf.cast(tf.not_equal(tensor, padding_value), 'int32'), axis = -1), 'float32')

In [0]:
def ctc_decoder(args):
  input_length, logits = args
  logits = tf.transpose(logits, perm = [1, 0, 2])
  decoded_sequences, _ = tf.nn.ctc_greedy_decoder(logits, tf.cast(input_length, 'int32'))
  decoded_sequences_dense = tf.sparse_to_dense(decoded_sequences[0].indices, decoded_sequences[0].dense_shape, decoded_sequences[0].values, default_value = -1)

  return tf.cast(decoded_sequences_dense, 'float32')

In [0]:
def tf_dense_to_sparse(dense, padding_value = 0):
  padding = tf.constant(padding_value, dtype = tf.float32)
  where = tf.not_equal(dense, padding)
  indices = tf.where(where)
  values = tf.gather_nd(dense, indices)
  return tf.SparseTensor(indices, values, tf.cast(tf.shape(dense), 'int64'))

In [0]:
def edit_distance_v2(y_true, y_pred):
  
  y_true_sparse = tf_dense_to_sparse(y_true, -1)
  y_pred_sparse = tf_dense_to_sparse(y_pred, -1)
  
  edit_distance = tf.edit_distance(y_pred_sparse, y_true_sparse, normalize = False)
  
  y_true_lengths = get_tensor_lengths(y_true, -1)
  y_pred_lengths = get_tensor_lengths(y_pred, -1)
  
  edit_distance = edit_distance / tf.maximum(y_true_lengths, y_pred_lengths)
  
  return 1.0 - tf.reduce_mean(edit_distance)

In [0]:
def _edit_distance_acc(input_length):
  
  def edit_distance_acc(y_true, y_pred):
    
    y_pred = ctc_decoder((input_length, y_pred))
    y_true = tf.squeeze(y_true, axis = -1)
    return edit_distance_v2(y_true, y_pred)
  
  return edit_distance_acc

In [0]:
def _ctc_loss(input_length):
  
  def ctc_loss(y_true ,y_pred):
    y_true = tf.squeeze(y_true, axis = -1)
    y_true = tf_dense_to_sparse(y_true, padding_value = -1)
    return tf.nn.ctc_loss(tf.cast(y_true, 'int32'), y_pred, tf.cast(input_length, 'int32'), time_major = False)

  return ctc_loss

In [0]:
def get_training_model(inputs, logit):

  input_length = Input(shape = (), name = 'input_length')
  training_model = Model([inputs, input_length], [logit])
  adam_optimizer = tf.keras.optimizers.Adam(lr = 0.0005)
  training_model.compile(loss = _ctc_loss(input_length), metrics = [_edit_distance_acc(input_length)], optimizer = adam_optimizer)
  
  return training_model

In [0]:
def get_prediction_model(inputs, logit):
    
  input_length = Lambda(get_input_length)(inputs)
  decoded_sequences = Lambda(ctc_decoder)([input_length, logit])
  prediction_model = Model(inputs, decoded_sequences)
  
  return prediction_model

In [0]:
def get_models():
  inputs, logit = create_model()
  training_model = get_training_model(inputs, logit)
  prediction_model = get_prediction_model(inputs, logit)
  
  return training_model, prediction_model

In [0]:
training_model, prediction_model = get_models()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


#Load Model/Optimizer state

In [0]:
def load_model(model = training_model, path = model_path):
  model.load_weights(path + 'weights.h5')
  #loading optimizer state to retrieve running means used in trainning
#   model._make_train_function()
  with open(path + 'optimizer.pkl', 'rb') as f:
      weight_values = pickle.load(f)
  model.optimizer.set_weights(weight_values)

In [0]:
try:
  load_model()
except:
  pass

#Save Model/Optimizer state



In [0]:
def save_model(model = training_model, path = model_path):
  model.save_weights(path + 'weights.h5')
  symbolic_weights = getattr(model.optimizer, 'weights')
  weight_values = K.batch_get_value(symbolic_weights)
  with open(path + 'optimizer.pkl', 'wb') as f:
      pickle.dump(weight_values, f)

In [0]:
class ModelSaver(keras.callbacks.Callback):
  def __init__(self, filepath = model_path, period = 1):
    self.period = period
    self.filepath = filepath    
    
  def on_train_begin(self, logs = {}):
      self.best_acc = float('-inf')
      self.count = 0
        
  def on_epoch_end(self, batch, logs = {}):    
    self.count += 1
    current_acc = logs['val_edit_distance_acc']
       
    if(self.count % self.period == 0):
      if(current_acc > self.best_acc):
        print("epoch number: ", self.count, "saved")
        self.best_acc = current_acc
        save_model(path = self.filepath)

#Train Model

In [0]:
gc.collect()

0

In [0]:
def generator(X_batches, Y_batches, input_length_batches):
    while True:     
        for X_batch, Y_batch, input_length_batch in zip(X_batches, Y_batches, input_length_batches):
          yield ([X_batch,  input_length_batch], Y_batch)

In [0]:
#training_model.fit([X_train_batches[273][16:17], input_length_batches[273][16:17], label_length_batches[273][16:17]], Y_train_batches[273][16:17], epochs = 5, batch_size = 1, shuffle = False)

In [0]:
training_model.fit_generator(generator(X_train_batches, Y_train_batches, input_length_train), 
                             steps_per_epoch = len(X_train_batches), epochs = epochs, callbacks = [ModelSaver(period = 2)],
                             validation_data = generator(X_test_batches, Y_test_batches, input_length_test), validation_steps = len(X_test_batches), 
                             max_queue_size = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
 4916/11897 [===========>..................] - ETA: 29:42 - loss: inf - edit_distance_acc: 0.7153

In [0]:
#model.fit([tf_data, np.concatenate(input_length_batches), np.concatenate(label_length_batches)], tf_labels, epochs = 10, batch_size = 128)

# Random

In [0]:
prediction_model.save('prediction_model.h5')

In [0]:
max_audio_length = 100

In [0]:
#converter = tf.lite.TFLiteConverter.from_keras_model_file("prediction_model.h5", input_shapes = {"inputs" : [None, max_audio_length, input_dimension]})
# converter = tf.lite.TFLiteConverter.from_keras_model(prediction_model)
# tflite_model = converter.convert()
# open("prediction_model.tflite", "wb").write(tflite_model)

In [0]:
# #input is a list containing numpy arrays[samples], use none for dynamic padding. 
# def tf_padding(input_list, element_shape, batch_size):
  
#   def custom_generator():  
#     while True:
#         for item in input_list:
#           yield item
#         break
        
#   tf_data = tf.data.Dataset.from_generator(custom_generator, output_types = tf.float64)
#   tf_data = tf_data.padded_batch(batch_size = batch_size, padded_shapes = element_shape)
#   iterator = tf_data.make_one_shot_iterator()
#   data = iterator.get_next()
#   tf_batches = []

#   with tf.Session() as sess:
#     while True:
#       try:
#         tf_batches.append(sess.run(data))
#       except tf.errors.OutOfRangeError:
#         break
        
#   return tf_batches
  

In [0]:
# tf_data = np.concatenate(tf_padding(X, tf.TensorShape([600, input_dimension]), 1))
# tf_labels = np.expand_dims(np.concatenate(tf_padding(Y, tf.TensorShape([200]), 1)), axis = 2)

In [0]:
def edit_distance(arr1, arr2):
  n = arr1.shape[0]
  m = arr2.shape[0]
  dp = np.zeros((n + 1, m + 1), dtype = int)
  for i in range(n + 1):
    for j in range(m + 1):
      if i==0:
        dp[i][j] = j
      elif j==0:
        dp[i][j] = i
      elif arr1[i-1] == arr2[j - 1]:
        dp[i][j] = dp[i - 1][j - 1]
      else:
        dp[i][j] = 1 + min(dp[i][j - 1], dp[i - 1][j], dp[i - 1][j - 1])
  return  1 - dp[n][m] / max(n, m)

In [0]:
index2byte = ['<EMP>', ' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g',
              'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q',
              'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [0]:
def index2str(index_list):
    # transform label index to character
    str_ = ''
    for ch in index_list:
        if ch > 0:
            str_ += index2byte[ch]
        else:  # <EOS>
            break
    return str_

In [0]:
def print_index(indices):
    for index_list in indices:
        print(index2str(index_list))

In [0]:
def print_predictions():
  for j in range(len(X_test_batches)):
    for i in range(1, batch_size):
      y_true = Y_test_batches[j][i - 1 : i]
      x_test = X_test_batches[j][i - 1 : i]
      input_length = input_length_test[j][i - 1 : i]
      loss, metric = training_model.evaluate([x_test, input_length], y_true, batch_size = batch_size, verbose = 0)      
      y_pred = prediction_model.predict(x_test, batch_size = batch_size).astype('int32')
      print('ctc loss = %f, edit distance = %f' %(loss, metric))
      print_index(np.squeeze(y_true, axis = -1))
      print_index(y_pred)

In [0]:
print_predictions()